# **Clone Repo and install dependencies**

In [ ]:
!git clone https://github.com/ultralytics/yolov5
!pip install -qr yolov5/requirements.txt
%cd yolov5

import torch
from IPython.display import Image, clear_output
clear_output()

# Download custom dataset

In [ ]:
#dataset custom facut de mine, l-am pus in google drive
!gdown https://drive.google.com/uc?id=1ezuBbiAjEUJwUmTy5rUDSULfwSItQDDV&export=download
!unzip /content/Fire_Detector.zip

# **Use a custom model configuration**

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: 2  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
# Import PyYAML module
import yaml

# Define the YAML content as a dictionary
data = {'train': '/content/Fire_Detector/Dataset/train/images',
        'val': '/content/Fire_Detector/Dataset/train/images',
        'nc': 2,
        'names': ['smoke', 'fire']} #ar trebui sa detecteze atat focul, cat si fumul

# Set the file path and name
file_path = '/content/data.yaml'

# Open the file in write mode and write the YAML content
with open(file_path, 'w') as file:
    documents = yaml.dump(data, file)

In [ ]:
#convert our pascal voc to yolov5, because the dataset is not compatible by the default, I had a mental breakdown here
!pip install pylabel


In [ ]:
from pylabel import importer
dataset = importer.ImportVOC(path="/content/Fire_Detector/Dataset/train/labels")
dataset.export.ExportToYoloV5(output_path="/content/Fire_Detector/Dataset/train/labels") 

Exporting YOLO files...: 100%|██████████| 2603/2603 [00:05<00:00, 505.01it/s]


['/content/Fire_Detector/Dataset/train/dataset.yaml',
 '/content/Fire_Detector/Dataset/train/labels/ckatz7phaxvp30744g7atzcpp.txt',
 '/content/Fire_Detector/Dataset/train/labels/ckavq3ufjaav60753v4hu04p1.txt',
 '/content/Fire_Detector/Dataset/train/labels/ckb2luyeof6mq0797vzj2d3qf.txt',
 '/content/Fire_Detector/Dataset/train/labels/ckakcgymw9p9y0b1299w8x1pe.txt',
 '/content/Fire_Detector/Dataset/train/labels/ckb3mifj0059o0755mc4u2zii.txt',
 '/content/Fire_Detector/Dataset/train/labels/ckau25v54yhgt072176o500hp.txt',
 '/content/Fire_Detector/Dataset/train/labels/ckb33hv5x04gt0768gjzz6wh4.txt',
 '/content/Fire_Detector/Dataset/train/labels/ckau1pp0vy9i507535h93lwy4.txt',
 '/content/Fire_Detector/Dataset/train/labels/img (72).txt',
 '/content/Fire_Detector/Dataset/train/labels/ckandspwb5fcy08531whqdsnf.txt',
 '/content/Fire_Detector/Dataset/train/labels/ckb2lujlmf6gi0751kdfvt8cs.txt',
 '/content/Fire_Detector/Dataset/train/labels/ckb2lj4eq00gp07747q6wxj8b.txt',
 '/content/Fire_Detector/Da

In [ ]:
dataset = importer.ImportVOC(path="/content/Fire_Detector/Dataset/validation/labels")
dataset.export.ExportToYoloV5(output_path="/content/Fire_Detector/Dataset/validation/labels") 

Exporting YOLO files...: 100%|██████████| 90/90 [00:00<00:00, 640.76it/s]


['/content/Fire_Detector/Dataset/validation/dataset.yaml',
 '/content/Fire_Detector/Dataset/validation/labels/small (23).txt',
 '/content/Fire_Detector/Dataset/validation/labels/img (24).txt',
 '/content/Fire_Detector/Dataset/validation/labels/pic (14).txt',
 '/content/Fire_Detector/Dataset/validation/labels/small (15).txt',
 '/content/Fire_Detector/Dataset/validation/labels/small (7).txt',
 '/content/Fire_Detector/Dataset/validation/labels/pic (29).txt',
 '/content/Fire_Detector/Dataset/validation/labels/img (20).txt',
 '/content/Fire_Detector/Dataset/validation/labels/img (3).txt',
 '/content/Fire_Detector/Dataset/validation/labels/pic (15).txt',
 '/content/Fire_Detector/Dataset/validation/labels/img (26).txt',
 '/content/Fire_Detector/Dataset/validation/labels/pic (2).txt',
 '/content/Fire_Detector/Dataset/validation/labels/small (17).txt',
 '/content/Fire_Detector/Dataset/validation/labels/pic (19).txt',
 '/content/Fire_Detector/Dataset/validation/labels/pic (12).txt',
 '/content/F

# **Train on new dataset**

In [ ]:
# train yolov5s on custom data for 10 epochs(10 generatii), pentru acuratete mai mare, putem pune mai multe
# time its performance
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 16 --epochs 10 --data '../data.yaml' --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

/content/yolov5
train: weights=, cfg=./models/custom_yolov5s.yaml, data=../data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
requirements: /content/requirements.txt not found, check failed.
YOLOv5 🚀 v7.0-162-gc3e4e94 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0

# **Test on Images and Videos**

# New Section

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/yolov5s_results9/weights/best.pt --img 416 --conf 0.4 --source /content/Fire_Detector/Dataset/validation/images

detect: weights=['/content/yolov5/runs/train/yolov5s_results9/weights/best.pt'], source=/content/Fire_Detector/Dataset/validation/images, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: /content/requirements.txt not found, check failed.
YOLOv5 🚀 v7.0-162-gc3e4e94 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7249215 parameters, 0 gradients
image 1/90 /content/Fire_Detector/Dataset/validation/images/img (1).jpg: 256x416 1 fire, 41.0ms
image 2/90 /content/Fire_Detector/Dataset/validation/images/img (10).jpg: 320x416 1 fire, 42.8ms
image 3/90 /content/Fire_Detector/D

In [ ]:
import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/inference/output/*.jpg'):
    display(Image(filename=imageName))
    print("\n")